# Introducción 

# Objetivos 

El objetivo general de la prueba `recommender_system_test` es evaluar el impacto de un sistema de recomendaciones mejorado en el comportamiento de los usuarios en términos de conversión a través del embudo de pago. La prueba involucra dos grupos: el grupo de control (A) y el grupo experimental con el nuevo embudo de pago (B). La prueba se lanzó el 7 de diciembre de 2020, dejó de aceptar nuevos usuarios el 21 de diciembre de 2020 y finalizó el 1 de enero de 2021. La audiencia de la prueba consiste en el 15% de los nuevos usuarios de la región de la UE, con un número previsto de 6,000 participantes. El propósito de la prueba es medir si, dentro de los 14 días posteriores a la inscripción, los usuarios muestran una mejor conversión en vistas de la página del producto (`product_page`), en la acción de agregar artículos al carrito de compras (`product_cart`) y en las compras (`purchase`). Se espera que en cada etapa del embudo (`product_page → product_cart → purchase`), haya al menos un aumento del 10%. Los datos de la prueba deben descargarse, comprobarse para asegurar que la prueba se realizó correctamente, y analizarse para evaluar los resultados obtenidos.


# Descomposición de la tarea

* ...

# Carga de librerías 

In [4]:
# Carga de librerías 
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import graph_objects as go

# Carga de datos

In [5]:
# Carga de datasets desde carpeta local
data_path = os.path.join("..", "datasets")

dataset1_path = os.path.join(data_path, "ab_project_marketing_events_us.csv")
dataset2_path = os.path.join(data_path, "final_ab_events_upd_us.csv")
dataset3_path = os.path.join(data_path, "final_ab_new_users_upd_us.csv")
dataset4_path = os.path.join(data_path, "final_ab_participants_upd_us.csv")

ab_project_marketing_data = pd.read_csv(dataset1_path)
final_ab_events_data = pd.read_csv(dataset2_path)
final_ab_new_events_data = pd.read_csv(dataset3_path)
final_ab_participants_data = pd.read_csv(dataset4_path)

In [6]:
# Carga de datos en jupyter de platforma

"""ab_project_marketing_data = pd.read_csv("/datasets/ab_project_marketing_events_us.csv")
final_ab_events_data = pd.read_csv("/datasets/final_ab_events_upd_us.csv")
final_ab_new_events_data = pd.read_csv("/datasets/final_ab_new_users_upd_us.csv")
final_ab_participants_data = pd.read_csv("/datasets/final_ab_participants_upd_us.csv")"""

'ab_project_marketing_data = pd.read_csv("/datasets/ab_project_marketing_events_us.csv")\nfinal_ab_events_data = pd.read_csv("/datasets/final_ab_events_upd_us.csv")\nfinal_ab_new_events_data = pd.read_csv("/datasets/final_ab_new_users_upd_us.csv")\nfinal_ab_participants_data = pd.read_csv("/datasets/final_ab_participants_upd_us.csv")'

## Exploración inicial de datos

### Revisión de dataset ab_project_marketing_data

In [7]:
# Revisión de datos ab_project_data
print(ab_project_marketing_data.head(5))
print()
print(ab_project_marketing_data.info())

                           name                   regions    start_dt  \
0      Christmas&New Year Promo             EU, N.America  2020-12-25   
1  St. Valentine's Day Giveaway  EU, CIS, APAC, N.America  2020-02-14   
2        St. Patric's Day Promo             EU, N.America  2020-03-17   
3                  Easter Promo  EU, CIS, APAC, N.America  2020-04-12   
4             4th of July Promo                 N.America  2020-07-04   

    finish_dt  
0  2021-01-03  
1  2020-02-16  
2  2020-03-19  
3  2020-04-19  
4  2020-07-11  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes
None


- `name`: el nombre del evento de marketing
- `regions`: regiones donde se llevará a cabo la campaña publicitaria
- `start_dt`: fecha de inicio de la campaña
- `finish_dt`: fecha de finalización de la campaña

In [8]:
# Cambio de formato de columna start_dt
ab_project_marketing_data['start_dt'] = pd.to_datetime(ab_project_marketing_data['start_dt'])
ab_project_marketing_data['finish_dt'] = pd.to_datetime(ab_project_marketing_data['finish_dt'])

### Revisión de dataset final_ab_events_data

In [9]:
# Revisión de dataset final_ab_events_data
print(final_ab_events_data.head())
print()
print(final_ab_events_data.info())

            user_id             event_dt event_name  details
0  E1BDDCE0DAFA2679  2020-12-07 20:22:03   purchase    99.99
1  7B6452F081F49504  2020-12-07 09:22:53   purchase     9.99
2  9CD9F34546DF254C  2020-12-07 12:59:29   purchase     4.99
3  96F27A054B191457  2020-12-07 04:02:40   purchase     4.99
4  1FD7660FDF94CA1F  2020-12-07 10:15:09   purchase     4.99

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB
None


- `user_id`
- `event_dt`: fecha y hora del evento
- `event_name`: nombre del tipo de evento
- `details`: datos adicionales sobre el evento (por ejemplo, el pedido total en USD para los eventos `purchase`)

In [10]:
# Cambio de formato de columna event_date
final_ab_events_data['event_dt'] = pd.to_datetime(final_ab_events_data['event_dt'])

In [11]:
# Revisión de datos en la columna 'details'
final_ab_events_data['details'].sample(10)

235704      NaN
367812      NaN
417395      NaN
3070      99.99
349011      NaN
405491      NaN
241930      NaN
21208      4.99
152206      NaN
241037      NaN
Name: details, dtype: float64

In [12]:
# Revisión de datos que contienen 'NaN' en la columna 'details'
no_purchase = final_ab_events_data[final_ab_events_data['details'].isna()]
no_purchase.sample(10)

# Se aplica reemplazo de valores NaN por 0 
final_ab_events_data.fillna({'details':0}, inplace = True)
print(final_ab_events_data.sample(10))

                 user_id            event_dt    event_name  details
45652   E387BE3E42D68438 2020-12-22 22:56:59      purchase     4.99
242124  C8ABA13613426B05 2020-12-07 07:44:21         login     0.00
293189  1DEACD4780374FF4 2020-12-14 19:29:24         login     0.00
392581  1862E009C8A4DACD 2020-12-23 04:39:07         login     0.00
63222   AC7C385CBEBC2D78 2020-12-08 02:06:16  product_cart     0.00
252441  B3F6B623A4E96DD5 2020-12-09 14:59:48         login     0.00
363360  8EC85ACFA6A394FA 2020-12-21 10:04:42         login     0.00
124661  7125F88F8C1A757E 2020-12-08 16:36:24  product_page     0.00
3985    6C68D47386054C43 2020-12-09 17:03:31      purchase     4.99
391604  BDFD5B22807A1DCF 2020-12-23 07:53:59         login     0.00


In [13]:
# Verificación de los valores de eventos
final_ab_events_data['event_name'].value_counts()

event_name
login           182465
product_page    120862
purchase         60314
product_cart     60120
Name: count, dtype: int64

Columna 'details' contiene información numérica y NaN, la documentación del dataset indica que tiene la cantidad en USD para cuando el evento es 'purchase'. Se infiere que los datos NaN son 0, ya que no representan una transacción concretada. 
Se observa que hay más valores de `purchase` que de `product_cart`, lo que puede sugerir que se evita esta última etapa y se puede comprar con algo parecido a un botón 'compra rápida'

### Revisión de dataset final_ab_new_events_data

In [14]:
# Revisión de datos en dataset final_ab_new_events_data
print(final_ab_new_events_data.sample(10))
print()
print(final_ab_new_events_data.info())

                user_id  first_date     region   device
58141  DEBDF3B09AA48037  2020-12-20  N.America   iPhone
27502  7DDE747C9C024FF2  2020-12-09         EU   iPhone
43664  CC5E51CFA52E8463  2020-12-12         EU       PC
43232  8C5967A3EC28773A  2020-12-18         EU  Android
26325  087DFCB9E3F7BCF0  2020-12-09         EU   iPhone
1753   9F4B7C7D64845781  2020-12-07         EU   iPhone
49068  69FEB4629D5A8BBB  2020-12-19         EU       PC
13453  B7B62610D29DEF22  2020-12-21         EU   iPhone
48184  B63BB5743CB1BC70  2020-12-19  N.America  Android
43681  0902526564A6C13C  2020-12-12         EU  Android

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8

- `user_id`
- `first_date`: fecha de inscripción
- `region`
- `device`: dispositivo utilizado para la inscripción

In [15]:
# Se cambia el formato de la columna 'first_date' 
final_ab_new_events_data['first_date'] = pd.to_datetime(final_ab_new_events_data['first_date'])


No contiene datos nulos y el único cambio realizado fue el formato de fecha en la columna 'first_date'

### Revisión del dataset final_ab_participants_data

In [16]:
# Revisión del dataset 

print(final_ab_participants_data.head())
print()
print(final_ab_participants_data.info())

            user_id group                  ab_test
0  D1ABA3E2887B6A73     A  recommender_system_test
1  A7A3664BD6242119     A  recommender_system_test
2  DABC14FDDFADD29E     A  recommender_system_test
3  04988C5DF189632E     A  recommender_system_test
4  4FF2998A348C484F     A  recommender_system_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB
None


- `user_id`
- `ab_test`: nombre de la prueba
- `group`: el grupo de prueba al que pertenecía el usuario

# Segmentación de datos para realizar test A/B

Se requieren los datos que cumplan con las siguientes condiciones:
* Usuarios que se hayan inscrito entre 7 y 18 de diciembre. 
* Que pertenezcan a la región de UE
* 